In [ ]:
import requests, pickle, zipfile, dict2xml, xmltodict, os, shutil, re, petrovna
from pathlib import Path
from datetime import date, timedelta, datetime
import datetime as dt
import numpy as np
from dateutil.relativedelta import relativedelta
import pandas as pd

#--------Calculating dates and filenames
today=date.today()
year=(today.replace(day=1) - timedelta(days=1)).year
year2=str(year%100)
month = (today.replace(day=1) - timedelta(days=1)).month#-----------------------------<=Check month!
month2=f"{month:02d}"
code_mo="660017"
package_number="01"#------------------------------------------------------------------------------<=Check package!
filename_h="H"+code_mo+"_"+year2+month2+package_number
filename_l="L"+code_mo+"_"+year2+month2+package_number
print(filename_h,filename_l)

#---------All pathes here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
anaconda_work_dir=r'C:\Users\User\Anaconda\\'
download_dir=r'D:\downloads\\'
shared_work_dir=r'S:\10 - Статистика\лаба\\'+str(year)+'\\'
ds_scheme_mapping_file=str(year)+'-'+month2+r'_ds-schema-ksg.ods'
rg_ct_file=shared_work_dir+str(year)+'-'+month2+r'_ct-rg.xlsx'
pcr_gepatity_file=shared_work_dir+str(year)+'-'+month2+r'_ГИБ-HCV-ПЦР.xlsx'
ifa_gepatity_file=shared_work_dir+str(year)+'-'+month2+r'_ГИБ-HCV-ИФА.xlsx'

baseurl="https://ecp.mis66.ru"
api_person = baseurl + "/api/Person"
api_person_list_url = baseurl + "/api/PersonList"
api_police = baseurl + "/api/Polis"
api_usluga = baseurl + "/api/EvnUsluga"
api_usluga_par = baseurl + "/api/EvnUslugaPar"
api_usluga_list = baseurl + "/api/EvnUslugaList"
api_visit = baseurl + "/api/EvnVizitPL"
ecp_searchData = baseurl + "/?c=Search&m=searchData"
ecp_tap_url=baseurl + '/?c=EvnPL&m=loadEvnPLEditForm'
ecp_visit_url=baseurl + '/?c=EvnVizit&m=loadEvnVizitPLEditForm'
ecp_delete_usluga_url=baseurl + '/?c=EvnUsluga&m=deleteEvnUsluga'
ecp_load_usluga_url=baseurl + '/?c=EvnUslugaPar&m=loadEvnUslugaParEditForm'
ecp_person_url=baseurl + '/?c=Person&m=getPersonEditWindow'

cookie_file=Path("cookies.txt")
os.chdir(anaconda_work_dir)

pcr_gepatity_ready=False
pcr_covid_ready=False
fibros_ready=False
cts_ready=False

code_pcr_gepatitis_quality='A26.05.019.001.101' #  694 rub
code_pcr_gepatitis_genotype='A26.05.019.003' # 949/954 rub
code_ifa_gepatitis='A26.06.041.002'
#code_ifa_gepatitis='A26.06.041'
code_pcr_covid='A26.08.008.102'            #   508/514 rub
code_ct_lungs='A06.09.005' #                      1424 rub
code_ct_head='A06.23.004' #                       rub  
code_ct_abdomen='A06.30.005' #                    rub
code_ct_abdomen_aorta_contrast='A06.12.001.002' # 4060 rub
code_rg= 'A06.09.007.101' #                        567 rub
code_fibroscan='A04.14.001.005' #                  881 rub
code_cancels='B01.014.001.222' #                   900 rub
code_tmks='B01.014.002.888' #                      826 rub
code_app_primary='B01.014.001' #                   978 rub
code_app_repeat='B01.014.002' #                    652 rub
per_snils=""
per_polis=""
fibroscan_allowed_diagnoses=['K70.0', 'K70.1', 'K70.2', 'K70.3', 'K70.4', 'K70.9', 'K71.7', 'K72.0', 'K72.1', 'K72.9', 'K74.0', 'K74.1', 'K74.2', 'K74.3', 'K74.4', 'K74.5', 'K74.6', 'K76.0', 'K76.7', 'E83', 'E83.0', 'E83.1', 'E83.2', 'E83.3', 'E83.4', 'E83.5', 'E83.8', 'E83.9', 'B18.0', 'B18.1', 'B18.2', 'B18.8', 'B18.9']
def ecp_polis_from_snils(snils):
    payload = { 'Person_Snils': snils, 'limit': '1' }
    response = session.request("GET", baseurl+'/?c=Person&m=getPersonSearchGrid', params=payload).json()
    ecp_polis=response['data'][0]['Polis_Num']
    return ecp_polis

def yesterday_range(date=today):
    yesterday = today - timedelta(days=1)
    yesterday_string = yesterday.strftime("%d.%m.%Y")
    return yesterday_string+" - " + yesterday_string

def month_range(year=datetime.now().year,month=datetime.now().month):
    range_start=date(year,month,1).strftime("%d.%m.%Y")
    range_end=(date(year,month,1)+relativedelta(months=1)-timedelta(days=1)).strftime("%d.%m.%Y")   
    return range_start+" - " + range_end
    
def enhanced_month_range(year=datetime.now().year,month=datetime.now().month):
    range_start=(date(year,month,1)-timedelta(days=1)).strftime("%d.%m.%Y")
    range_end=(date(year,month,1)+relativedelta(months=1)).strftime("%d.%m.%Y")
    return range_start+" - " + range_end

def load_cookies():
    print("Loading cookies")
    if cookie_file.is_file():
        print("Cookies found!")
        with open(cookie_file, 'rb') as f:
            session.cookies.update(pickle.load(f))
    else:
        print("Cookies not found")

def save_cookies():
    print("Saving cookies")
    with open(cookie_file, 'wb') as f:
       pickle.dump(session.cookies, f)
    
def login():
    print("Logining")
    headers = {
            "Login": os.getenv('ECP_USERNAME_1'),
            "Password": os.getenv('ECP_PASSWORD_1')
    }
    response=session.request("GET", f"{baseurl}/api/user/login", headers=headers).json()
    save_cookies
    print(response)
    
def sex_is_magic(patronymic):
    if type(patronymic) != str:
        return 2
    if len(patronymic)<4:
        return 2
    if (patronymic[-3:]=='вич' or patronymic[-4:]=='оглы' or patronymic[-4:]=='угли'):
        return 1 
    return 2

def get_person_id_by_snils(snils):
    response = session.request("GET", api_person, params={"PersonSnils_Snils": snils})
    json_response = response.json()
    return json_response['data'][0]['Person_id']

def get_person_by_id(person_id):
    response = session.request("GET", api_person, params={"Person_id": person_id})
    return response.json()

def get_police_by_id(police_id):
    response = session.request("GET", api_police, params={"Polis_id": police_id})
    return response.json()

def find_polices_by_snils():
    for index, row in table.iterrows():
        person_snils=row['СНИЛС']
        if len(person_snils)>10:
            person_id=get_person_id_by_snils(person_snils)
            row['ID']=[person_id]
            person_police=get_person_by_id(person_id)['data'][0]['Polis_Num']
        else:
            personId="Error"
        print(person_snils+","+person_id+","+person_police)
        
def ecp_find_kss_out_daterange(date_range):
    payload = {'PersonPeriodicType_id': '1', 'SearchFormType': 'EvnPS', 'Date_Type':'1', 'EvnPS_disDate_Range':date_range}
    #payload['LpuUnitType_did']=1#FOR_KSS_ONLY:
    print('hello')
    response = session.request("GET", ecp_searchData, params=payload)
    return response.json()

def ecp_find_kss_in_daterange(date_range):
    payload = {'PersonPeriodicType_id': '1', 'SearchFormType': 'EvnPS', 'Date_Type':'1', 'EvnPS_setDate_Range':date_range}
    payload['LpuUnitType_did']=1##FOR_KSS_ONLY:
    print('hello')
    response = session.request("GET", ecp_searchData, params=payload)
    return response.json()

def search_vizit(tap_id):
    payload = {
        'EvnPLBase_id' : tap_id
        }
    return session.request("GET", api_visit, params=payload).json()

def delete_usluga(usluga_id):
    payload = {
        'class' : 'EvnUslugaCommon',
        'id' : usluga_id
        }
    return session.request("POST", ecp_delete_usluga_url, params=payload).json()

def search_usluga(tap_id):
    return session.request("GET", api_usluga_list, params={'Evn_pid' : tap_id, }).json()

def get_usluga(usluge_id):
    return session.request("GET", api_usluga, params={'EvnUsluga_id' : usluge_id, }).json()

def ecp_search_fibroscan_usluga(date_range):
    payload = {
        'PersonPeriodicType_id' : '1', 
        'SearchFormType' : 'EvnUslugaPar', 
        'UslugaComplex_id' : '226103', 
        'EvnUslugaPar_setDate_Range' : date_range,
        'SearchType_id' : '1', 
        'Part_of_the_study' : 'false', 
        'PersonCardStateType_id' : '1', 
        'PrivilegeStateType_id' : '1', 
        'limit' : '1000', 
    }
    return session.request("GET", ecp_searchData, params=payload).json()

def ecp_load_usluga(uslugapar_id):
    return session.request("POST", ecp_load_usluga_url, params={'EvnUslugaPar_id' : uslugapar_id}).json()

def ecp_get_person(person_id):
    return session.request("POST", ecp_person_url, params={'person_id' : person_id}).json()

def ecp_search_person_by_police(police):
    return session.request("POST", baseurl + '/?c=Person&m=getPersonSearchGrid', params={'Polis_Num' : police}).json()

def print_data_for_generating_fibroscan(date_range):
    ecp_search_fibroscan_usluga_response=ecp_search_fibroscan_usluga(date_range)
    for record in ecp_search_fibroscan_usluga_response['data']:
        pacient_list.append(record['Person_id'])
    for pacient in pacient_list:
        ecp_get_person_resp=ecp_get_person(pacient)
        person_id=ecp_get_person_resp[0]['Person_id']
        ecp_get_person_resp=ecp_get_person(person_id)[0]
        #ecp_get_person_resp
        if 'Federal_Num' in ecp_get_person_resp:
            pacient_polis=ecp_get_person_resp['Federal_Num']
            if len(pacient_polis)<14:
                    print('old police!!!')
        else:
            print('Нет полиса!!!!!!!!!!')
        pacient_name=ecp_get_person_resp['Person_FirName']
        pacient_surname=ecp_get_person_resp['Person_SurName']
        pacient_patronomyc=ecp_get_person_resp['Person_SecName']
        pacient_sex=ecp_get_person_resp['PersonSex_id']
        pacient_bithdate=datetime.strptime(ecp_get_person_resp['Person_BirthDay'],"%d.%m.%Y").strftime("%Y-%m-%d")
        if 'Person_SNILS' in ecp_get_person_resp:
            pacient_snils=ecp_get_person_resp['Person_SNILS']
        else:
            pacient_snils=''
        print(pacient_polis,'\t',pacient_surname,'\t',pacient_name,'\t',pacient_patronomyc,'\t',pacient_sex,'\t',pacient_bithdate,'\t',pacient_snils)

def ecp_get_direction(year):
    return session.request("POST", baseurl + '/?c=EvnDirection&m=getEvnDirectionNumber', 
                           params={'year' : year}).json()
def ecp_add_ct(direction,date,diag,pacient_id,pacient_evn_id,server_id):
    payload = {
        'Org_did': '1007',
        'EvnDirection_setDate' : date,
        'PrehospDirect_id' : 1,
        'Lpu_did' : 0,
        'LpuSection_did': '99560025390',
        'LpuSection_uid': '99560025390',
        'EvnUslugaPar_id' : '0',
        'EvnUslugaPar_pid' : '0',
        'EvnUslugaPar_isCito' : '0',
        'TimetablePar_id' : '0',
        'EvnDirection_id' : '0',
        'Person_id':pacient_id,
        'PersonEvn_id' : pacient_evn_id,
        'Server_id' : server_id,
        'EvnUslugaPar_IsWithoutDirection' : '1',
        'PrehospDirect_id' : '1',
        'EvnDirection_Num' : direction,
        'MedPersonal_uid' : '1708',
        'MedPersonal_did' : '1708', #вроде Христова
        'MedStaffFact_uid' : '63118',
        'MedStaffFact_did' : '63118', #вроде Христова
        'EvnUslugaPar_MedPersonalCode' : '3719',
        'UslugaPlace_id' : '1',
        'LpuSectionProfile_id' : '401089',
        'EvnUslugaPar_setDate' : date,
        'EvnUslugaPar_setTime' : '12:34',
        'EvnUslugaPar_disDate' : date,
        'EvnUslugaPar_disTime' : '12:34',
        'UslugaCategory_id' : '68',
        'UslugaComplex_id' : '4903417',
        'FSIDI_id' : '7001894',
        'Diag_id' : diag,
        'DeseaseType_id' : '3',
        'Mes_id' : '962665',
        'PayType_id' : '110',
        'EvnUslugaPar_Kolvo' : '1',
        }
    return session.request("POST", baseurl + '/?c=EvnUslugaPar&m=saveEvnUslugaPar', params=payload).json()

def add_ct(ct_list):
    for ct in ct_list:
        direction=ecp_get_direction(year)['EvnDirection_Num']
        date=ct['date']
        diag='J18.9'#ct['ds']
        police=ct['police']
        ecp_search_person_by_police_resp=ecp_search_person_by_police(police)['data'][0]
        pacient_id=ecp_search_person_by_police_resp['Person_id']
        pacient_evn_id=ecp_search_person_by_police_resp['PersonEvn_id']
        server_id=ecp_search_person_by_police_resp['Server_id']    
        ecp_add_ct_resp=ecp_add_ct(direction,date,diag,pacient_id,pacient_evn_id,server_id)
        ecp_add_ct_resp

def generate_zap_per_app(zap_count, 
                     usl_count, 
                     history_num,
                     date,
                     npr_mo,
                     npr_doctor,
                     podr, 
                     profil,
                     prvs,
                     iddokt,
                     ds,
                     code_usl, 
                     npolis,
                     fam,
                     im,
                     ot,
                     w,
                     dr,
                     npr_num = None,
                     npr_date = None,
                     c_zab=1,
                     snils="",
                     vpolis=3, 
                     smo="66004", 
                     p_cel=12,
                     razdel_usl=301,
                     kol_usl=1,
                     usl_ok=3,
                     vidpom=13,
                     npr_type=4,
                     idsp=28,
                     rslt=314,
                     ishod=304,
                     for_pom=3,
                     okato="65476000000",
                     mes=0,
                    ):
    if npr_num is None:
        npr_num=history_num
        print ('npr_num is None')
    if npr_date is None:
        npr_date=date
        print ('npr_date is None')
    pacient={"ID_PAC":zap_count,
        "VPOLIS":vpolis,
        "NPOLIS":npolis,
        "SMO":smo,
        "NOVOR":"0",
        "SOC":"000"} # 1-дошкольник, 2-учащийся, 3-работающий, 4-неработающий, 5-пенсионер
    usl={"IDSERV":usl_count,
        "LPU":code_mo,
        "PODR":podr,
        "PROFIL":profil,
        "P_CEL":p_cel,
        "DATE_IN":date,
        "DATE_OUT":date,
        "DS":ds,
        "RAZDEL_USL":razdel_usl,
        "CODE_USL":code_usl,
        "KOL_USL":kol_usl,
        "MR_USL_N":{"MR_N":1,
            "PRVS":prvs,
            "CODE_MD":iddokt,
            "MAIN_MD":1
            },
        }
    sluch={"IDCASE":zap_count,
        "USL_OK":usl_ok,
        "VIDPOM":vidpom,
        "NPR_MO":npr_mo,
        "NPR_DOCTOR":npr_doctor,
        "NPR_TYPE":npr_type,
        "NPR_NUM":npr_num,
        "NPR_DATE":npr_date,
        "DS_ONK":"0",
        "PODR":podr,
        "PROFIL":profil,
        "NHISTORY":history_num,
        "DATE_1":date,
        "DATE_2":date,
        "C_ZAB":c_zab,
        "DS1":ds,
        "RSLT":rslt,
        "ISHOD":ishod,
        "PRVS":prvs,
        "IDDOKT":iddokt,
        "IDSP":idsp,
        "FOR_POM":for_pom,
        "USL":[usl]}
    if mes!=0:
        sluch["CODE_MES1"]=mes
    zap={"N_ZAP":zap_count,
            "PR_NOV":"0",
            "PACIENT":pacient,
            "SLUCH":sluch}
    per={"ID_PAC":zap_count,
        "FAM":fam,
        "IM":im,
        "OT":ot,
        "W":w,
        "DR":dr,
        "SNILS":snils,
        "OKATOG":okato,
        "OKATOP":okato}
    if ot == '':
        del per['OT']
        per['DOST']='1'
    return zap, per

def parse_snils(snils):
    if snils == '' or snils is None or snils == 0:
        return ''
    if type(snils)==str:
        list_snils = re.findall(r'\d+', snils)
        snils = ''.join(list_snils)
    if type(snils)==int:
        snils="{:011d}".format(snils)
    return snils[:3]+"-"+snils[3:6]+"-"+snils[6:9]+" "+snils[9:]

def calculate_age_in_years(birth_date_str, reference_date=None):
    if reference_date is None:
        reference_date = dt.datetime.now().date()
    elif isinstance(reference_date, str):
        reference_date = parse_date(reference_date)
    elif isinstance(reference_date, dt.datetime):
        reference_date = reference_date.date()
    
    # Parse birth date
    if isinstance(birth_date_str, (dt.date, dt.datetime)):
        birth_date = birth_date_str.date() if isinstance(birth_date_str, dt.datetime) else birth_date_str
    else:
        birth_date = parse_date(birth_date_str)
    
    # Calculate age
    age = reference_date.year - birth_date.year
    
    # Adjust age if birthday hasn't occurred yet this year
    if (reference_date.month, reference_date.day) < (birth_date.month, birth_date.day):
        age -= 1
        
    return age

def parse_date(date_str):
    if not isinstance(date_str, str):
        raise TypeError("Date must be a string")
    
    # Try Russian format (dd.mm.yyyy)
    if re.match(r'^\d{1,2}\.\d{1,2}\.\d{4}$', date_str):
        try:
            return dt.datetime.strptime(date_str, '%d.%m.%Y').date()
        except ValueError:
            pass
    
    # Try ISO format and other common formats using dateutil parser
    try:
        return parser.parse(date_str).date()
    except (ValueError, parser.ParserError):
        raise ValueError(f"Could not parse date: {date_str}")

date_range=month_range(year,month)
enhanced_date_range=enhanced_month_range(year,month)
session = requests.session()
load_cookies()
login()
print('year=',year,'  month=',month,'  date_range=',date_range,'  enhanced_date_range=',enhanced_date_range)

H660017_250701 L660017_250701
Loading cookies
Cookies found!
Logining
{'error_code': 0, 'sess_id': 'feh0avdgvj9njfhkjtam76ms5l'}
year= 2025   month= 7   date_range= 01.07.2025 - 31.07.2025   enhanced_date_range= 30.06.2025 - 01.08.2025


In [ ]:
#---------------------add rg to taps from taps number /запустить один раз, после добавления всех ТАПов/ 
def delete_usluga_from_taps(tap_list):
    if type(tap_list) is not list: tap_list = [tap_list]
    count=0
    for tap in tap_list:
        ecp_find_tap_by_num_response = ecp_find_tap_by_num(tap).json()
        tap_id=ecp_find_tap_by_num_response['data'][0]['EvnPL_id']
        search_usluga_response=search_usluga(tap_id)
        for record in search_usluga_response['data']:
            if 'EvnUsluga_id' in record:
                usluga_id=search_usluga_response['data'][0]['EvnUsluga_id']
                count+=1
                print(count, 'found usluga', usluga_id, 'in', tap, '. Trying to delete:')
                delete_usluga_response=delete_usluga(usluga_id)
                delete_usluga_response
            else:
                print('there are no usluga in', tap)

def ecp_find_tap_by_num(num):
    payload = {
        'PersonPeriodicType_id' : '1', 
        'SearchFormType' : 'EvnPL', 
        'EvnPL_NumCard' : num, 
        'Vizit_Date_Range' : enhanced_date_range,
        'Date_Type' : '1', 
        'AddressStateType_id' : '1', 
        'PersonCardStateType_id' : '1', 
        'EvnPS_disDate_Range' : enhanced_date_range}
    return session.request("GET", ecp_searchData, params=payload).json()

def ecp_add_rg(pid,setdt,disdt,diag):
    payload = {
        'Evn_pid': pid, #идентификатор случая–родителя (движение в КВС, посещения в ТАП, ПОСЕЩЕНИЕ!)
        'EvnClass_id' : 22,
        'Evn_setDT' : setdt,
        'Evn_disDT' : disdt,
        'UslugaPlace_id': 1,
        'LpuSection_id': '99560031796',#(N, У/О) ; 99560017798 - ПП; 99560031796 - РГ
        'Lpu_id' : '',#(N, У/О) 
        'LpuSectionProfile_id' : '401089',#(N, У/О) – профиль отделения МО; 401048 - ПП; 401089 - РГ
        'MedStaffFact_id' : '151056',#(N, У/О) – место работы врача, оказавшего услугу; 62646 - плёнкин в пп; 151056 - Чалкова в РГ
        'PayType_id' : 110,#(N, О) – вид оплаты;
        'DiagSetClass_id' : 1,#(N, О) – тип диагноза;
        'Diag_id':diag,#(N, О) – диагноз;
        'UslugaComplex_id' : '9990004110',#(N, О) – услуга ; '9990004110' - РГ
        'EvnUsluga_Kolvo' : 1,#(N, О) – количество;
        'MedPersonal_id' : '1708', # 51656 - Плёнкин; 1708 - Христова
        'BeamLoad_RadiationDose':100,
        'BeamLoadUnits_id':1,
        }
    return session.request("POST", api_usluga, params=payload).json()

def add_rg_to_taps(tap_list):
    if type(tap_list) is not list: tap_list = [tap_list]
    count=0
    for tap in tap_list:
        ecp_find_tap_by_num_response=ecp_find_tap_by_num(tap)
        tap_id=ecp_find_tap_by_num_response['data'][0]['EvnPL_id']
        search_vizit_response=search_vizit(tap_id)
        search_vizit_response['data']
        visit_id=search_vizit_response['data'][0]['EvnVizitPL_id']
        visit_date=search_vizit_response['data'][0]['Evn_setDT']
        visit_diag=search_vizit_response['data'][0]['Diag_id']
        count+=1
        print(count, 'tap', tap, ', visit_id', visit_id, ', visit_date', visit_date, ', visit_diag', visit_diag,)
        ecp_add_rg_response=ecp_add_rg(visit_id,visit_date,visit_date,visit_diag)
        print(ecp_add_rg_response)

def ecp_get_usluga(usluga_id):
    payload = {
        'class' : 'EvnUslugaCommon',
        'pid' : usluga_id,
        'parent' : 'EvnPL',
        }
    return session.request("POST", baseurl + '/?c=EvnUsluga&m=loadEvnUslugaGrid', params=payload).json()
        
def find_usluga_in_taps(tap_list):
    if type(tap_list) is not list: tap_list = [tap_list]
    count=0
    for tap_num in tap_list:
        ecp_find_tap_by_num_response=ecp_find_tap_by_num(tap_num)
        tap_id=ecp_find_tap_by_num_response['data'][0]['EvnPL_id']
        ecp_get_usluga_response=ecp_get_usluga(tap_id)
        usluga_id=ecp_get_usluga_response[0]['EvnUsluga_id']
        print(usluga_id)
        usluga_list.append(usluga_id)
        
rg_list=[]
rg_table=pd.read_excel(rg_ct_file, sheet_name='export-rg')
for index, row in rg_table.iterrows():
    rg_list.append(row['NUMCARD'])
print(rg_list)

add_rg_to_taps(rg_list)

print(datetime.now())

In [ ]:


#--------------------->>> RESTART HERE <<<----------------------#



In [4]:
#-------------------------------/A/0. ds schemes (добавление лекарств в ДС в xml)
import xml.etree.ElementTree as ET

def add_schemes_to_records(xml_file_path, history_scheme_mapping, scheme_definitions):
    """
    Add LEK_PR schemes to specific medical records based on history numbers.
    
    Parameters:
    - xml_file_path: Path to the input XML file
    - history_scheme_mapping: Dictionary mapping history numbers to scheme codes
    - scheme_definitions: Dictionary containing the details for each scheme
    """
    # Parse the XML file
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    # Find all ZAP records
    zap_records = root.findall('./ZAP')
    print(f"Found {len(zap_records)} ZAP records")
    
    # Process each ZAP record
    temp_count=0
    for zap in zap_records:
        # print('zap:', end=' ')
        # print(zap.find('./N_ZAP').text, end='; ')
        # Find NHISTORY element
        scluch = zap.find('./SLUCH')
        if scluch is None:
            print(zap.find('./N_ZAP').text,'scluch is None')
            continue
            
        nhistory_elem = scluch.find('./NHISTORY')
        if nhistory_elem is None:
            print(zap.find('./N_ZAP').text,'nhistory_elem is None')
            continue
            
        nhistory = nhistory_elem.text
        print(nhistory, end='; ')
        # Check if this history number needs a scheme
        if nhistory in history_scheme_mapping:
            print(zap.find('./N_ZAP').text,'NHISTORY is in mapping!', end='; ')
            temp_count+=1
            scheme_code = history_scheme_mapping[nhistory]
            print('scheme_code=',scheme_code, end='; ')
            # Check if the scheme exists in definitions
            if scheme_code not in scheme_definitions:
                print(f"Warning: Scheme {scheme_code} not found in definitions for history {nhistory}")
                continue
                
            scheme_info = scheme_definitions[scheme_code]
            print('scheme_info=',scheme_info, end='; ')
            # Find USL elements to get IDSERV and DATE_IN
            usl_elements = zap.findall('./SLUCH/USL')
            if not usl_elements:
                print(f"Warning: No USL elements found for history {nhistory}")
                continue
                
            # Get the first USL's IDSERV and DATE_IN (can be modified to select specific USL)
            usl = usl_elements[0]
            idserv_elem = usl.find('./IDSERV')
            date_in_elem = usl.find('./DATE_IN')
            
            if idserv_elem is None or date_in_elem is None:
                print(f"Warning: Missing IDSERV or DATE_IN for history {nhistory}")
                continue
                
            idserv = idserv_elem.text
            date_in = date_in_elem.text
            
            # Get regnum list for this scheme
            regnum_list = scheme_info['regnum']
            if not isinstance(regnum_list, list):
                regnum_list = [regnum_list]
            
            # Add LEK_PR elements for each regnum in the scheme
            for regnum in regnum_list:
                # Create LEK_PR element
                lek_pr = ET.SubElement(scluch, 'LEK_PR')
                
                # Add required elements
                ET.SubElement(lek_pr, 'IDSERV').text = idserv
                ET.SubElement(lek_pr, 'LEK_P').text = scheme_info.get('lek_p', '3')
                ET.SubElement(lek_pr, 'CODE_SH').text = scheme_code
                ET.SubElement(lek_pr, 'REGNUM').text = str(regnum)
                ET.SubElement(lek_pr, 'REG_D').text = scheme_info.get('reg_d', '0')
                ET.SubElement(lek_pr, 'REG_A').text = scheme_info.get('reg_a', '1')
                ET.SubElement(lek_pr, 'REG_V').text = scheme_info.get('reg_v', '1')
                
                # Create INJ element
                inj = ET.SubElement(lek_pr, 'INJ')
                ET.SubElement(inj, 'DATE_INJ').text = date_in
                ET.SubElement(inj, 'ED_IZM').text = scheme_info.get('ed_izm', '10')
                ET.SubElement(inj, 'METHOD_INJ').text = scheme_info.get('method_inj', '2')
                
                # Add scheme-specific values
                kv_value = scheme_info.get('kv_values', {}).get(regnum, '500.000')
                ET.SubElement(inj, 'KV_INJ').text = kv_value
                ET.SubElement(inj, 'KIZ_INJ').text = kv_value
                ET.SubElement(inj, 'S_INJ').text = scheme_info.get('s_inj', '1.00')
                
                # Calculate SV_INJ and SIZ_INJ based on KV_INJ
                sv_value = f"{float(kv_value):.2f}"
                ET.SubElement(inj, 'SV_INJ').text = sv_value
                ET.SubElement(inj, 'SIZ_INJ').text = sv_value
                ET.SubElement(inj, 'RED_INJ').text = '0' # <<< --------------------------- добавить
                ET.SubElement(inj, 'COL_INJ').text = scheme_info.get('col_inj', '1')
            
            print(f"Added scheme {scheme_code} for history {nhistory}")
    
    # Write the modified XML to output file
    tree.write(xml_file_path, encoding='utf-8', xml_declaration=True)
    print(f"Modified XML saved to {xml_file_path}. {temp_count} records modified.")

def read_history_scheme_mapping(mappinf_file):
    """
    Creates a dictionary from two columns in an ODS file.

    Returns:
    dict: Dictionary with '№ карты' as keys and 'Схема' as values
    """
    try:
        # Read the ODS file
        df = pd.read_excel(mappinf_file, engine='odf')
        
        # Check if both columns exist in the dataframe
        if '№ карты' not in df.columns or 'Схема' not in df.columns:
            missing = []
            if '№ карты' not in df.columns:
                missing.append('№ карты')
            if 'Схема' not in df.columns:
                missing.append('Схема')
            raise KeyError(f"Columns {', '.join(missing)} not found in the file")
        
        # Create a dictionary from the two columns with keys converted to strings
        result_dict = {str(key): value for key, value in zip(df['№ карты'], df['Схема'])}
        
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return None
        
def directions_by_person_id(person_id):
    payload = {'Person_id': person_id}
    response = session.request("GET", baseurl + "/api/EvnDirectionList", params=payload).json()
    return response

def direction_by_id(direction_id):
    payload = {'EvnDirection_id': direction_id}
    response = session.request("GET", baseurl + "/api/EvnDirection", params=payload).json()
    return response

def cancel_direction_by_id(direction_id):
    payload = {'EvnDirection_id': direction_id}
    response = session.request("PUT", baseurl + "/api/EvnDirectionCancel", params=payload).json()
    return response

def get_lpu_by_id(lpu_id):
    url=rbaseurl + "/api/Refbook/Lpu"
    payload = {'Lpu_id':lpu_id}
    response = session.request("GET", url, params=payload).json()
    print('----------(response[data][0])------------')
    print(response['data'][0])
    return response['data'][0]

def get_person_by_id(person_id):
    url=rbaseurl + "/api/Person"
    payload = {'Person_id':person_id}
    response = session.request("GET", url, params=payload).json()
    return response['data'][0]
    
def get_medpersonal_by_id(medpersonal_id):
    url=rbaseurl + "/api/MedWorkerById"
    payload = {'MedWorker_id':medpersonal_id}
    response1 = session.request("GET", url, params=payload).json()['data']
    response2 = get_person_by_id(response1['Person_id'])
    response2['CodeDLO']=response1['CodeDLO']
    return response2

def search_ifa_direction_for_reestr_by_person_id(person_id):
    directions_by_person_id_resp=directions_by_person_id(person_id=person_id)
    for direction in list(reversed(directions_by_person_id_resp['data'])):
        direction_by_id_resp=direction_by_id(direction_id=direction['EvnDirection_id'])['data'][0]
        if direction_by_id_resp['LpuSection_did']=='99560026173': # and direction_by_id_resp['MedService_Name']=='Иммунология':
            # print(direction_by_id_resp)
            medpersonal=get_medpersonal_by_id(direction_by_id_resp['MedPersonal_id'])
            # print(medpersonal)
            lpu=get_lpu_by_id(direction_by_id_resp['Lpu_sid'])
            # print(lpu)
            answer={
                'npr_num':direction_by_id_resp['EvnDirection_Num'],
                'npr_date':direction_by_id_resp['EvnDirection_setDate'],
                'npr_mo':lpu['Lpu_f003mcod'], 
                'npr_doctor':medpersonal['CodeDLO'],
            }
            print(answer)
            return answer
    print('search_ifa_direction_for_reestr_by_person_id is failed!!!') 
    
scheme_definitions = {
                    'thc01': {
                        'regnum': '2536',
                        'lek_p': '3',
                        'reg_d': '0',
                        'reg_a': '1',
                        'reg_v': '1',
                        'ed_izm': '10',
                        'method_inj': '2',
                        'kv_values': {'2536':'500.000'},
                        's_inj': '1.00',
                        'col_inj': '1'
                    },
                    'thc03': {
                        'lek_p': '3',
                        'regnum': '655',
                        'reg_d': '0',
                        'reg_a': '1',
                        'reg_v': '1',
                        'ed_izm': '10',
                        'method_inj': '2',
                        'kv_values': {'655':'420.000'},
                        's_inj': '1.00',
                        'col_inj': '1'
                    },
                    'thc16': {
                        'regnum': ['704', '1975'],
                        'lek_p': '3',
                        'reg_d': '0',
                        'reg_a': '1',
                        'reg_v': '1',
                        'ed_izm': '10',
                        'method_inj': '2',
                        'kv_values': {'704': '60.000', '1975': '400.000'},
                        's_inj': '1.00',
                        'col_inj': '1'
                    },
                    'thc17': {
                        'regnum': '1878',
                        'lek_p': '3',
                        'reg_d': '0',
                        'reg_a': '1',
                        'reg_v': '1',
                        'ed_izm': '10',
                        'method_inj': '2',
                        'kv_values': {'1878':'1000.000'},
                        's_inj': '1.00',
                        'col_inj': '1'
                    },
                }


# Define mapping of history numbers to scheme codes
history_scheme_mapping = read_history_scheme_mapping(shared_work_dir+ds_scheme_mapping_file)
print(history_scheme_mapping)

shutil.copy(download_dir+filename_h+'.zip', filename_h+'.zip')
with zipfile.ZipFile(filename_h+".ZIP") as zf:
    zf.extract(filename_h+'.XML')
    add_schemes_to_records(filename_h+'.XML', history_scheme_mapping, scheme_definitions)
print(datetime.now())
shutil.copy(filename_h+'.XML', download_dir+filename_h+'.XML')

# >>> ------------------- Отменить любое направление по его id -------------------------- <<<
# directions_by_person_id_resp=directions_by_person_id(343203)
# print(directions_by_person_id_resp['data'])
# cancel_direction_by_id_resp=cancel_direction_by_id(660910635743212)
# print(cancel_direction_by_id_resp)
# direction_by_id_resp=direction_by_id(660910635743212)
# print(direction_by_id_resp)

{'8541': 'thc01', '8481': 'thc01', '8487': 'thc01', '1521/': 'thc01', '8483': 'thc01', '8489': 'thc01', '8478': 'thc01', '8531': 'thc01', '6562': 'thc01', '7479': 'thc01', '8480': 'thc01', '8482': 'thc03', '8488': 'thc03', '8484': 'thc03', '8545': 'thc03', '8493': 'thc03', '8452': 'thc03', '8497': 'thc03', '8533': 'thc03', '8486': 'thc03', '8535': 'thc03', '8479': 'thc03', '8543': 'thc03', '8553': 'thc03', '8532': 'thc03', '8534': 'thc03', '8536': 'thc16', '8539': 'thc16', '8498': 'thc16', '8503': 'thc16'}
Found 684 ZAP records
1521/; 1 NHISTORY is in mapping!; scheme_code= thc01; scheme_info= {'regnum': '2536', 'lek_p': '3', 'reg_d': '0', 'reg_a': '1', 'reg_v': '1', 'ed_izm': '10', 'method_inj': '2', 'kv_values': {'2536': '500.000'}, 's_inj': '1.00', 'col_inj': '1'}; Added scheme thc01 for history 1521/
6562; 2 NHISTORY is in mapping!; scheme_code= thc01; scheme_info= {'regnum': '2536', 'lek_p': '3', 'reg_d': '0', 'reg_a': '1', 'reg_v': '1', 'ed_izm': '10', 'method_inj': '2', 'kv_valu

'D:\\downloads\\\\H660017_250701.XML'

In [19]:
#----------------->>> /A/1. Read zip

shutil.copy(download_dir+filename_h+'.zip', filename_h+'.zip')
with zipfile.ZipFile(filename_h+".ZIP") as zf:
    h_xml = zf.open(zf.namelist()[0])
    l_xml = zf.open(zf.namelist()[1])
    h = xmltodict.parse(h_xml)
    l = xmltodict.parse(l_xml)
    l_xml.close()
    h_xml.close()
print('было:',len(h['ZL_LIST']['ZAP']) if not isinstance(h['ZL_LIST']['ZAP'], dict) else 1,len(l['PERS_LIST']['PERS']) if not isinstance(h['ZL_LIST']['ZAP'], dict) else 1)
print(datetime.now())

было: 1 1
2025-07-08 22:37:53.838587


In [17]:
h

{'ZL_LIST': {'ZGLV': {'VERSION': '4.1',
   'DATA': '2025-07-08',
   'FILENAME': 'H660017_250207',
   'SD_Z': '1'},
  'SCHET': {'CODE_MO': '660017',
   'YEAR': '2025',
   'MONTH': '2',
   'DEVELOPER': '6'},
  'ZAP': {'N_ZAP': '1',
   'PR_NOV': '0',
   'PACIENT': {'ID_PAC': '1',
    'VPOLIS': '3',
    'NPOLIS': '6655520877000713',
    'SMO': '66063',
    'NOVOR': '0'},
   'SLUCH': {'IDCASE': '1',
    'USL_OK': '2',
    'VIDPOM': '13',
    'NPR_MO': '660017',
    'NPR_DOCTOR': '34343',
    'NPR_TYPE': '2',
    'NPR_NUM': '11122',
    'NPR_DATE': '2025-02-25',
    'DS_ONK': '0',
    'PODR': '60000',
    'PROFIL': '28',
    'NHISTORY': '1521/',
    'P_PER': '5',
    'DATE_1': '2025-02-26',
    'DATE_2': '2025-02-26',
    'KD': '1',
    'DS1': 'B18.2',
    'RSLT': '201',
    'ISHOD': '202',
    'PRVS': '24',
    'IDDOKT': '34343',
    'IDSP': '33',
    'FOR_POM': '3',
    'USL': {'IDSERV': '1',
     'LPU': '660017',
     'PODR': '60000',
     'PROFIL': '28',
     'PROFIL_K': '24',
     'DATE

In [20]:
#------------------------------------>>>/AB/2. The beginning of xml formation. Reseting zaps'n'pers<<<-------------------------------------
zaps, pers=[], []
zap_count=10000
usl_count=10000

In [19]:
#------------------------------------>>>/AB/3. The beginning of xml formation. Fibroscan<<<-------------------------------------

# fibros_ready=False
if not fibros_ready:
    zaps_fibroscan, pers_fibroscan=[], []
    fibros=ecp_search_fibroscan_usluga(date_range)
    for fibro in fibros['data']:
        fibro_id=fibro['EvnUslugaPar_id']
        usluga=ecp_load_usluga(fibro_id)[0]
        pacient_id=usluga['Person_id']
        person=ecp_get_person(pacient_id)[0]
        print(person['Person_SurName'], end=', ')
        date=datetime.strptime(usluga['EvnUslugaPar_setDate'],'%d.%m.%Y').date().isoformat()
        if type(person['Person_SecName']) == str:
            patronymic=person['Person_SecName'].strip()
        else:
            patronymic=person['Person_SecName'] = ''
        #get_police_by_id_resp=get_police_by_id(police_id)
        zap_count+=1
        usl_count+=1
        zap, per = generate_zap_per_app(zap_count, 
                                        usl_count, 
                                        history_num=fibro_id,
                                        date=date,
                                        npr_mo=code_mo,
                                        npr_doctor=41132,
                                        npr_num=(date+str(person['Federal_Num']))[:24],
                                        npr_date=date,
                                        podr='100000', 
                                        profil='106',
                                        prvs='81',
                                        iddokt='3719', #usluga['EvnUslugaPar_MedPersonalCode']
                                        ds='B18.2',
                                        c_zab=1,
                                        code_usl=usluga['UslugaComplex_Code'], 
                                        npolis=person['Federal_Num'],
                                        fam=person['Person_SurName'],
                                        im=person['Person_FirName'],
                                        ot=patronymic,
                                        w=person['PersonSex_id'],
                                        dr=datetime.strptime(person['Person_BirthDay'],'%d.%m.%Y').date().isoformat(),
                                        snils=parse_snils(person['Person_SNILS']),
                                        vpolis=int(person['PolisType_id'])-1, 
                                        smo="66004", 
                                        p_cel=12,
                                        razdel_usl=301,
                                        kol_usl=1,
                                        usl_ok=3,
                                        vidpom=13,
                                        npr_type=4,
                                        idsp=28,
                                        rslt=314,
                                        ishod=304,
                                        for_pom=3,
                                        okato="65476000000"
                                       )

        pers_fibroscan.append(per)
        zaps_fibroscan.append(zap)
        print("added fibro:", zap_count)
    fibros_ready=True
else:
    zap_count+=len(zaps_fibroscan)
    usl_count+=len(zaps_fibroscan)
pers.extend(pers_fibroscan)
zaps.extend(zaps_fibroscan)
print('Добавлено фибросканов:',len(pers_fibroscan),len(zaps_fibroscan))
print(datetime.now())

ШАНТИБАЕВ, added fibro: 10001
ВИНОГРАДОВА, added fibro: 10002
БРИТМАН, added fibro: 10003
УСМАНОВ, added fibro: 10004
КИСЕЛЕВ, added fibro: 10005
НИКОЛАУ, added fibro: 10006
ВЕРЕТЕННИКОВА, added fibro: 10007
ПИНЯГИН, added fibro: 10008
ЯКУШИНА, added fibro: 10009
ИСАКОВ, added fibro: 10010
КАРТАВЫХ, added fibro: 10011
КАРНИЦКАЯ, added fibro: 10012
ВОЛОСЕВИЧ, added fibro: 10013
НИКИТИН, added fibro: 10014
АЛМАЕВА, added fibro: 10015
АМИРОВА, added fibro: 10016
ГИЛЕВ, added fibro: 10017
КУЗНЕЦОВА, added fibro: 10018
ХЛЕБНИКОВ, added fibro: 10019
МИХЕЕВА, added fibro: 10020
МАЛИКОВА, added fibro: 10021
СУЛЬДИНА, added fibro: 10022
МАЛЫГИНА, added fibro: 10023
ВЕЛИЧКОВА, added fibro: 10024
САВИН, added fibro: 10025
СИМОНОВА, added fibro: 10026
ПАНЬКОВА, added fibro: 10027
ЩУКИН, added fibro: 10028
МОРОЗОВ, added fibro: 10029
ШТАЕРТ, added fibro: 10030
ПАНКАГИР, added fibro: 10031
МИРОНОВ, added fibro: 10032
РАЗДОБАРИНА, added fibro: 10033
ГОРБАНЬ, added fibro: 10034
БАРАНОВА, added fibro: 

In [45]:
#-------------------------------------- /AB/4. add cts to reg
# cts_ready=False
if not cts_ready:
    zaps_ct, pers_ct=[], []
    per_polis=''
    ct_table=pd.read_excel(rg_ct_file, sheet_name='export-ct', converters={'polis' : str } )
    for index, row in ct_table.iterrows():
        print("try to add ct:", row['polis'], end=', ')
        person=ecp_search_person_by_police(row['polis'])['data'][0]
        #date=datetime.strptime(usluga['EvnUslugaPar_setDate'],'%d.%m.%Y').date().isoformat()
        if type(person['PersonSecName_SecName']) == str:
            patronymic=person['PersonSecName_SecName'].strip()
        else:
            patronymic=person['PersonSecName_SecName'] = ''
        zap_count+=1
        usl_count+=1
        if row['type']=='кт':
            ct_ds='J18.9'
            ct_usluga=code_ct_lungs
            ct_type='ct_lungs'
        elif row['type']=='кт гм':
            ct_ds='I64'
            ct_usluga=code_ct_head
            ct_type='ct_head'
        elif row['type']=='кт обп':
            ct_ds='K52.9'
            ct_usluga=code_ct_abdomen
            ct_type='ct_abdomen'
        elif row['type']=='кт обп контр':
            ct_ds='I71.4'
            ct_usluga=code_ct_abdomen_aorta_contrast
            ct_type='ct_abdctr'
        else:
            print(' UNKNOWN TARGET ',end=', ')
        date=row['date'].to_pydatetime().date()
        history_num=(date.strftime('%Y%m%d')+ct_type+str(zap_count))[:24]
#        history_num=(date.strftime('%Y%m%d')+ct_type+person['Person_id'])[:24]
        print(zap_count, history_num)
        if row['polis'] == per_polis:
            flag_skip=False
            #Если такая услуга уже есть (дубликат), то игнорируем:
            for u in zaps_ct[-1]['SLUCH']['USL']:
                if u['CODE_USL']==ct_usluga:
                    flag_skip=True
            if flag_skip:
                print("----------------found dup, skipping------------------")
                continue
            print("----------------adding second usluga------------------")
            usl_count+=1
            usl={"IDSERV":usl_count,
                "LPU":code_mo,
                "PODR":'100000',
                "PROFIL":'78',
                "P_CEL":12,
                "DATE_IN":date,
                "DATE_OUT":date,
                "DS":ct_ds,
                "RAZDEL_USL":301,
                "CODE_USL":ct_usluga,
                "KOL_USL":1,
                "MR_USL_N":{"MR_N":1,
                            "PRVS":'60',
                            "CODE_MD":'3719',
                            "MAIN_MD":1
                            },
                }
            zaps_ct[-1]['SLUCH']['USL'].append(usl)
            print("added usluga pcr:",per, zap['SLUCH']['USL'][1]['CODE_USL'])
            continue
        per_polis=row['polis']
        zap, per = generate_zap_per_app(zap_count, #N_ZAP IDCASE и IDPAC
                                        usl_count, #IDSERV
                                        history_num=history_num,
                                        date=date, #NPR_DATE, DATE_1, DATE_2, DATE_IN, DATE_OUT
                                        npr_mo=code_mo,
                                        npr_doctor=41130, #плёнкин 
                                        npr_num=history_num,
                                        npr_date=date,
                                        podr='100000', #Отделение лучевой диагностики
                                        profil='78', #Рентгенологии
                                        prvs='60',
                                        iddokt='3719', #Христова
                                        ds=ct_ds, #Пневма
                                        c_zab=1, #
                                        code_usl=ct_usluga, #КТ ГрОб
                                        npolis=person['Polis_EdNum'],
                                        fam=person['PersonSurName_SurName'],
                                        im=person['PersonFirName_FirName'],
                                        ot=patronymic,
                                        w=person['Sex_id'],
                                        dr=datetime.strptime(person['PersonBirthDay_BirthDay'],'%d.%m.%Y').date().isoformat(),
                                        snils=parse_snils(person['Person_Snils']),
                                        vpolis=3, 
                                        smo="66004", 
                                        p_cel=12,
                                        razdel_usl=301,
                                        kol_usl=1,
                                        usl_ok=3,
                                        vidpom=13,
                                        npr_type=4,
                                        idsp=28,
                                        rslt=314,
                                        ishod=304,
                                        for_pom=3,
                                        okato="65476000000",
                                        mes=1192,
                                       )
        pers_ct.append(per)
        zaps_ct.append(zap)
    cts_ready=True
else:
    zap_count=zaps_ct[-1]['N_ZAP']+1
    usl_count=zaps_ct[-1]['SLUCH']['USL'][-1]['IDSERV']+1
pers.extend(pers_ct)
zaps.extend(zaps_ct)
print('Добавлено КТ:',len(pers_ct),len(zaps_ct))
print(datetime.now())

try to add ct: 2151710880000283, 10001 20250509ct_lungs10001
try to add ct: 6650240893001109, 10002 20250501ct_lungs10002
10003 20250501ct_head1000300114, 
try to add ct: 6669260879000114, 10004 20250501ct_abdomen10004
----------------adding second usluga------------------
added usluga pcr: {'ID_PAC': 10003, 'FAM': 'ШАЛАГИНОВА', 'IM': 'РАИСА', 'OT': 'ФЕДОРОВНА', 'W': '2', 'DR': '1937-10-20', 'SNILS': '082-932-087 77', 'OKATOG': '65476000000', 'OKATOP': '65476000000'} A06.30.005
try to add ct: 6669260879000114, 10005 20250501ct_lungs10005
----------------adding second usluga------------------
added usluga pcr: {'ID_PAC': 10003, 'FAM': 'ШАЛАГИНОВА', 'IM': 'РАИСА', 'OT': 'ФЕДОРОВНА', 'W': '2', 'DR': '1937-10-20', 'SNILS': '082-932-087 77', 'OKATOG': '65476000000', 'OKATOP': '65476000000'} A06.30.005
10006 20250502ct_abdomen1000680, 
try to add ct: 6654510820002842, 10007 20250502ct_abdomen10007
10008 20250503ct_lungs100080753, 
try to add ct: 6655720819001502, 10009 20250503ct_lungs10009


In [ ]:
##### ------------------------------------>>>/AB/5. pcr_gepatity<<<-------------------------------------
pcr_gepatity_ready=False
flag_skip=False
if not pcr_gepatity_ready:
    zaps_gepatity, pers_gepatity=[], []

    pcr_gepatity=pd.read_excel(pcr_gepatity_file, sheet_name='Результат', converters={'СНИЛС' : str, '№ полиса' : str}, skiprows=8)# <- ПРОВЕРЬ МЕСЯЦ!!!!
    pcr_gepatity.dropna()

    #v13: Создаю в dataframe новую колонку date_all и заполняю её датами по формуле.
    pcr_gepatity['date_all'] = np.where(pcr_gepatity['Дата выполнения исследования (типирование HCV)'].isna(), pcr_gepatity['Дата выполнения исследования (качественная оценка содержания РНК HCV)'], pcr_gepatity['Дата выполнения исследования (типирование HCV)'])

    #Вернул проверку на месяц, не тестировал.
    #не заработало, забил пока.
    #pcr_gepatity=pcr_gepatity.loc[(pcr_gepatity['date_all'].dt.month == month)]

    #сортирую dataframe по вновь созданному столбцу и СНИЛС.
    pcr_gepatity = pcr_gepatity.sort_values(by=['date_all','СНИЛС'], ascending=True)
    
    for index, row in pcr_gepatity.iterrows():

        flag_vgs=False
        flag_vgs_genotip=False

        if type(row['Дата выполнения исследования (качественная оценка содержания РНК HCV)']) is str:
            #date_vgs=datetime.datetime.strptime(date1, '%d.%m.%Y').date().isoformat()
            #date_vgs=dt.datetime.strptime(row['Дата выполнения исследования (качественная оценка содержания РНК HCV)'], '%d.%m.%Y').date()
            flag_vgs=True
            #date_usl=date_vgs

        if type(row['Дата выполнения исследования (типирование HCV)']) is str:
            #date_vgs_genotip=datetime.datetime.strptime(date2, '%d.%m.%Y').date().isoformat()
            #date_vgs_genotip=dt.datetime.strptime(row['Дата выполнения исследования (типирование HCV)'], '%d.%m.%Y').date()
            flag_vgs_genotip=True
            #date_usl=date_vgs_genotip

        date_usl=dt.datetime.strptime(row['date_all'], '%d.%m.%Y').date()
                             
        for_count=1
        if flag_vgs and flag_vgs_genotip:
            print("----------------Две услуги!------------------")
            for_count=2
            if row['Дата выполнения исследования (типирование HCV)']!=row['Дата выполнения исследования (качественная оценка содержания РНК HCV)']:
                print ("Wrong dates?")
                raise SystemExit(1)

                
        for ii in range(for_count):
            
            date=date_usl.isoformat()
            #age=(row['Дата отчетности'].date()-row['Дата рождения'].date()).days
            age=(date_usl-dt.datetime.strptime(row['Дата рождения'], '%d.%m.%Y').date()).days
    
            #Пропускаем детей
            
            #if age<(365*18) and row['Название услуги'] == 'ВГС':
            if age<(365*18) and flag_vgs:
                print('-----> child! skipping quality test! <-------', age)
                continue
            #Проверяем СНИЛС:
    
            #v13: Странный формат в excel, при чтении пандой обрезает первый 0.
            #v13: добавляю вручную.
            snils=row['СНИЛС']
            snils=snils.replace("-","")
            snils=snils.replace(" ","")
            if len(snils)==10:
                snils='0'+snils
    
            if not petrovna.validate_snils(''.join([i for i in parse_snils(snils) if i.isdigit()])):
                print("-----------snils is bad----------0", snils)
                
            #Если пациент повторяется, то новый случай не создаём, только добавляем к последней записи ([-1]) ещё одну услугу.
            if parse_snils(snils) == per_snils:
                print('----------------Repeat! row:',index,'zap_count:',row['ФИО пациента'],zap_count)
                flag_skip=False
                #Если такая услуга уже есть (дубликат), то игнорируем:
                for u in zaps_gepatity[-1]['SLUCH']['USL']:
                    #Задумывалось как проверка на тип услуги и номер цикла, вроде как работает.
                    if u['CODE_USL']==(code_pcr_gepatitis_quality if ( flag_vgs and for_count==1 ) else code_pcr_gepatitis_genotype):
                        flag_skip=True
                if flag_skip:
                    print("----------------found dup, skipping------------------")
                    continue
                usl_count+=1
                usl={"IDSERV":usl_count,
                    "LPU":code_mo,
                    "PODR":'90000',
                    "PROFIL":'34',
                    "P_CEL":12,
                    "DATE_IN":date,
                    "DATE_OUT":date,
                    "DS":'B18.2',
                    "RAZDEL_USL":301,
                    "CODE_USL":code_pcr_gepatitis_quality if ( flag_vgs and for_count==1 ) else code_pcr_gepatitis_genotype,
                    "KOL_USL":1,
                    "MR_USL_N":{"MR_N":1,
                                "PRVS":'26',
                                "CODE_MD":'7229',
                                "MAIN_MD":1
                                },
                    }
                zaps_gepatity[-1]['SLUCH']['USL'].append(usl)
                print("added usluga pcr:",per, zap['SLUCH']['USL'][1]['CODE_USL'])
                continue
            
            zap_count+=1
            usl_count+=1
            per_snils = parse_snils(snils)

            fio=row['ФИО пациента']
    
            pos1=fio.find(' ')
            pos2=fio.find(' ',pos1+1)
            family=fio[0:pos1]
            name=fio[pos1+1:pos2]
            father=fio[pos2+1:len(fio)]
            
            if type(father) == str:
                patronymic=father.strip()
            else:
                patronymic = father = ''

            #Обработка пустых полисов из ЛИС
            if str(row['Полис'])=='nan':
                ecp_polis=ecp_polis_from_snils(per_snils)

                if ecp_polis=="":
                    print("-------------------------",family.strip(),name.strip(),patronymic," Полис в ЛИС пустой, полис в ЕЦП не найден !")
                    raise SystemExit(1)
        
                else:
                    print("-------------------------",family.strip(),name.strip(),patronymic," Полис в ЛИС пустой, полис из ЕЦП:", ecp_polis)
                
                row['Полис']=float(ecp_polis)

            history_num=(date+str(zap_count)+str(row['Полис']))[:24]
       
            zap, per = generate_zap_per_app(zap_count, 
                                            usl_count, 
                                            history_num=history_num,
                                            date=date,
                                            npr_mo=code_mo,
                                            npr_doctor='5644',
                                            npr_num=history_num,
                                            npr_date=date,
                                            podr='90000', 
                                            profil='34',
                                            prvs='26',
                                            iddokt='7229',
                                            ds='B18.2',
                                            code_usl=code_pcr_gepatitis_quality if flag_vgs else code_pcr_gepatitis_genotype,
                                            #v13: что-то пошло не так, преобразую в строку.
                                            npolis=str(int(row['Полис'])),
                                            fam=family.strip(),
                                            im=name.strip(),
                                            ot=patronymic,
                                            w=sex_is_magic(patronymic),
                                            dr=dt.datetime.strptime(row['Дата рождения'], '%d.%m.%Y').date().isoformat(),
                                            snils=per_snils,
                                            vpolis=3, 
                                            smo="66004", 
                                            p_cel=12,
                                            razdel_usl=301,
                                            kol_usl=1,
                                            usl_ok=3,
                                            vidpom=13,
                                            npr_type=4,
                                            idsp=28,
                                            rslt=314,
                                            for_pom=3,
                                            okato="65476000000",
                                           )
    
            pers_gepatity.append(per)
            zaps_gepatity.append(zap)
            print("added zap pcr-gepatit:",per, zap['SLUCH']['USL'][0]['CODE_USL'])

    
    pcr_gepatity_ready = True
pers.extend(pers_gepatity)
zaps.extend(zaps_gepatity)
print('Добавлено пцр-гепатитов:',len(pers_gepatity),len(zaps_gepatity))
print(datetime.now())

In [36]:
##### ------------------------------------>>>/AB/6. ifa_gepatity<<<-------------------------------------
ifa_gepatity_ready=False
if not ifa_gepatity_ready:
    zaps_ifa_gepatity, pers_ifa_gepatity=[], []
    ifa_gepatity=pd.read_excel(ifa_gepatity_file, sheet_name='Результат', converters={'СНИЛС' : str, '№ полиса' : str}, skiprows=7)# <- ПРОВЕРЬ МЕСЯЦ!!!!
    ifa_gepatity.dropna()

    #сортирую dataframe.
    ifa_gepatity = ifa_gepatity.sort_values(by=['Дата поступления материала','СНИЛС'], ascending=True)
    
    for index, row in ifa_gepatity.iterrows():

        #if type(row['Дата поступления материала']) is str:

        date_usl=dt.datetime.strptime(row['Дата поступления материала'], '%d.%m.%Y').date()
            
        date=date_usl.isoformat()
        age=(date_usl-dt.datetime.strptime(row['Дата рождения'], '%d.%m.%Y').date()).days

        #Проверяем СНИЛС:
        #v13: Странный формат в excel, при чтении пандой обрезает первый 0.
        #v13: добавляю вручную.
        snils=row['СНИЛС']
        snils=snils.replace("-","")
        snils=snils.replace(" ","")
        if len(snils)==10:
            snils='0'+snils

        if not petrovna.validate_snils(''.join([i for i in parse_snils(snils) if i.isdigit()])):
            print("-----------snils is bad----------0", snils)
            
        zap_count+=1
        usl_count+=1
        per_snils = parse_snils(snils)

        fio=row['ФИО пациента']

        pos1=fio.find(' ')
        pos2=fio.find(' ',pos1+1)
        family=fio[0:pos1]
        name=fio[pos1+1:pos2]
        father=fio[pos2+1:len(fio)]
        
        if type(father) == str:
            patronymic=father.strip()
        else:
            patronymic = father = ''

        #Обработка пустых полисов из ЛИС
        if str(row['Полис'])=='nan':
            ecp_polis=ecp_polis_from_snils(per_snils)

            if ecp_polis=="":
                print("-------------------------",family.strip(),name.strip(),patronymic," Полис в ЛИС пустой, полис в ЕЦП не найден !")
                raise SystemExit(1)
    
            else:
                print("-------------------------",family.strip(),name.strip(),patronymic," Полис в ЛИС пустой, полис из ЕЦП:", ecp_polis)
            
            row['Полис']=float(ecp_polis)

        history_num=(date+str(zap_count)+str(row['Полис']))[:24]
        napravlenie=search_ifa_direction_for_reestr_by_person_id(get_person_id_by_snils(per_snils))
        zap, per = generate_zap_per_app(zap_count, 
                                        usl_count, 
                                        history_num=history_num,
                                        date=date,
                                        npr_mo=napravlenie['npr_mo'],
                                        npr_doctor=napravlenie['npr_doctor'],
                                        podr='90000', 
                                        profil='34',
                                        prvs='26',
                                        iddokt='7229',
                                        ds='Z11.5',
                                        code_usl=code_ifa_gepatitis,

                                        npolis=str(int(row['Полис'])),
                                        fam=family.strip(),
                                        im=name.strip(),
                                        ot=patronymic,
                                        w=sex_is_magic(patronymic),
                                        dr=dt.datetime.strptime(row['Дата рождения'], '%d.%m.%Y').date().isoformat(),
                                        snils=per_snils,
                                        vpolis=3, 
                                        smo="66004", 
                                        p_cel=12,
                                        razdel_usl=301,
                                        kol_usl=1,
                                        usl_ok=3,
                                        vidpom=13,
                                        npr_num=napravlenie['npr_num'],
                                        npr_date=napravlenie['npr_date'],
                                        npr_type=4,
                                        idsp=28,
                                        rslt=314,
                                        for_pom=3,
                                        okato="65476000000",
                                        mes="7013-2" if w==1 else "7027-2",
                                       )
    
        pers_ifa_gepatity.append(per)
        zaps_ifa_gepatity.append(zap)
        print("added zap ifa-gepatit:", zap)
        # print("added zap ifa-gepatit:",per, zap['SLUCH']['USL'][0]['CODE_USL'])
    
    ifa_gepatity_ready = True
pers.extend(pers_ifa_gepatity)
zaps.extend(zaps_ifa_gepatity)
print('Добавлено ифа-гепатитов:',len(pers_ifa_gepatity),len(zaps_ifa_gepatity))
print(datetime.now())

----------(response[data][0])------------
{'Lpu_id': '13003613', 'Lpu_Name': 'государственное автономное учреждение здравоохранения Свердловской области "Городская больница № 1 город Нижний Тагил"', 'Lpu_f003mcod': '660010', 'LPU_OID': '1.2.643.5.1.13.13.12.2.66.6933'}
{'npr_num': '118073176', 'npr_date': '2025-04-29', 'npr_mo': '660010', 'npr_doctor': '47368'}


NameError: name 'w' is not defined

In [ ]:
#-------------------------------------->>>/AB/7. PCR COVID-19<<<------------------------------------- 
# pcr_covid_ready=False
if not pcr_covid_ready:
    # pcr_covid_file="S:\\10 - Статистика\\лаба\\2024\\2024-12_pcr-covid.ods"
    # pcr_covid_file="S:\\14 - Лаборатория\\списки обследованных на cov19\\обследованные на cov19 2024_.ods"
    pcr_covid_file="S:\\10 - Статистика\лаба\\2025\обследованные на cov19 2024.ods"
    pcr_covid=pd.read_excel(pcr_covid_file, sheet_name='Данные')
    pcr_covid.dropna()
    pcr_covid_selected = pcr_covid.loc[(pcr_covid['Название организации-заказчика'] != 'ГАУЗ СО ГИБ г.Н.Тагил')&(pcr_covid['Дата отчетности'].dt.month == month)&(pcr_covid['Дата отчетности'].dt.year == year)]
    pcr_covid_ready = True
print('было:',len(pers),len(zaps))
for index, row in pcr_covid_selected.iterrows():
    zap_count+=1
    usl_count+=1
    date=row['Дата отчетности'].date().isoformat()
    npolis=('%f'%row['ОМС']).rstrip('0').rstrip('.')
    code_npr_mo,code_npr_doctor='660010', '38945'
    if row['Название организации-заказчика']=='ГАУЗ СО ГБ №4':
        code_npr_mo,code_npr_doctor='660013', '7654'
    history_num=(date+npolis)[:24]
    if not petrovna.validate_snils(''.join([i for i in parse_snils(row['СНИЛС']) if i.isdigit()])):
        print("-----------snils is bad----------0", row['СНИЛС'])
    if type(row['Отчество']) == str:
        patronymic=row['Отчество'].strip()
    else:
        patronymic=row['Отчество'] = ''
    zap, per = generate_zap_per_app(
        zap_count, 
        usl_count, 
        history_num=history_num, 
        date=date, 
        npr_mo=code_npr_mo, 
        npr_doctor=code_npr_doctor, 
        podr='90000', 
        profil='34', 
        prvs='26', 
        iddokt='7229', 
        ds='J06.9', 
        code_usl=code_pcr_covid, 
        npolis=npolis, 
        fam=row['Фамилия'].strip(), 
        im=row['Имя'].strip(), 
        ot=patronymic, 
        w=sex_is_magic(patronymic), 
        dr=row['Дата рождения'].date().isoformat(), 
        snils=parse_snils(row['СНИЛС']), 
        vpolis=3, 
        smo="66004", 
        p_cel=12, 
        razdel_usl=301,
        kol_usl=1,
        usl_ok=3,
        vidpom=13, 
        npr_num=history_num, 
        npr_type=4,
        idsp=28,
        rslt=314,
        for_pom=3,
        okato="65476000000",
        mes=1101) 
    pers.append(per)
    zaps.append(zap)
    print("added pcr-grip:", zap, per)
print('стало:',len(pers),len(zaps))
print(datetime.now())

In [21]:
#------------------------------->>>/A/8. Append zaps and pers to xmls<<<----------------------

for zap in zaps:
    h['ZL_LIST']['ZAP'].append(zap)
for per in pers:
   l['PERS_LIST']['PERS'].append(per)

print('стало:',len(h['ZL_LIST']['ZAP']) if not isinstance(h['ZL_LIST']['ZAP'], dict) else 1,len(l['PERS_LIST']['PERS']) if not isinstance(h['ZL_LIST']['ZAP'], dict) else 1)

h['ZL_LIST']['ZGLV']['SD_Z']=len(h['ZL_LIST']['ZAP']) if not isinstance(h['ZL_LIST']['ZAP'], dict) else 1
print(datetime.now())

стало: 1 1
2025-07-08 22:38:47.594744


In [23]:
#-------------------------------->>>/A/9. write zip, sign, zip again and move to dowloads<<<----------------------

#filename_l=l['PERS_LIST']['ZGLV']['FILENAME']
#filename_h=l['PERS_LIST']['ZGLV']['FILENAME1']

with open(filename_h+".xml", "w") as f:
    f.write('<?xml version="1.0" encoding="Windows-1251"?>\n\n')
    f.write(dict2xml.Converter().build(data=h, data_sorter=dict2xml.DataSorter.never()))
with open(filename_l+".xml", "w") as f:
    f.write('<?xml version="1.0" encoding="Windows-1251"?>\n\n')
    f.write(dict2xml.Converter().build(data=l, data_sorter=dict2xml.DataSorter.never()))

with zipfile.ZipFile(filename_h+'.zip', 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zf:
    zf.write(filename_h+".xml", arcname=filename_h+".xml")
    zf.write(filename_l+".xml", arcname=filename_l+".xml")
os.system(r'cryptcp.exe -showstat -signf -detached -cert -thumbprint 2169dd12a7f5fca94bbaabaaf28d2d3bac06284c "'+filename_h+r'.zip"'+' -fext .sig')

with zipfile.ZipFile(filename_h+'.zip.sig.zip', 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zf:
    zf.write(filename_h+".zip", arcname=filename_h+".zip")
    zf.write(filename_h+".zip.sig", arcname=filename_h+".zip.sig")

shutil.move(filename_h+'.zip.sig.zip', download_dir+filename_h+'.zip.sig.zip')
os.remove(filename_h+".zip.sig")
os.remove(filename_h+".zip")
os.remove(filename_h+".xml")
os.remove(filename_l+".xml")
print(datetime.now())

2025-07-08 22:39:08.586305


In [47]:
#------------------------------->>>/B/10. Make full xml, write zip, sign, zip again and move to dowloads<<<-------------------------------

hxml={"ZL_LIST":{
    "ZGLV":{
        "VERSION":"3.2",
        "DATA":today,
        "FILENAME":filename_h,
        "SD_Z":len(zaps)},
    "SCHET":{
        "CODE_MO":code_mo,
        "YEAR":year,
        "MONTH":month,
        "DEVELOPER":"6",},
    "ZAP":zaps}}

lxml={
    "PERS_LIST":{
        "ZGLV":{
            "VERSION":"2.1",
            "DATA":today,
            "FILENAME":filename_l,
            "FILENAME1":filename_h,
        },
        "PERS":pers
    }
}

with open(filename_h+".xml", "w") as f:
    f.write('<?xml version="1.0" encoding="Windows-1251"?>\n\n')
    f.write(dict2xml.Converter().build(data=hxml, data_sorter=dict2xml.DataSorter.never()))
with open(filename_l+".xml", "w") as f:
    f.write('<?xml version="1.0" encoding="Windows-1251"?>\n\n')
    f.write(dict2xml.Converter().build(data=lxml, data_sorter=dict2xml.DataSorter.never()))
#with open("test.xml", "w") as f:
#    f.write(dict2xml.Converter().build(data=lxml, data_sorter=dict2xml.DataSorter.never()))
with zipfile.ZipFile(filename_h+'.zip', 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zf:
    zf.write(filename_h+".xml", arcname=filename_h+".xml")
    zf.write(filename_l+".xml", arcname=filename_l+".xml")
os.system(r'cryptcp.exe -showstat -signf -detached -cert -thumbprint 2169dd12a7f5fca94bbaabaaf28d2d3bac06284c "'+filename_h+r'.zip"'+' -fext .sig')

with zipfile.ZipFile(filename_h+'.zip.sig.zip', 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zf:
    zf.write(filename_h+".zip", arcname=filename_h+".zip")
    zf.write(filename_h+".zip.sig", arcname=filename_h+".zip.sig")

shutil.move(filename_h+'.zip.sig.zip', download_dir+filename_h+'.zip.sig.zip')
os.remove(filename_h+".zip.sig")
os.remove(filename_h+".zip")
os.remove(filename_h+".xml")
os.remove(filename_l+".xml")
print('ready!')
print(datetime.now())

ready!
2025-06-06 16:27:34.301741


In [ ]:
ecp_polis_from_snils("166-989-800 51")

In [ ]:
def napr_from_snils(snils):
    payload = { 'Person_Snils': snils, 'limit': '1' }
    #print(payload)
    response = session.request("GET", baseurl+'/?c=Person&m=getPersonSearchGrid', params=payload).json()
    ecp_polis=response['data'][0]['Polis_Num']
    return ecp_polis